Problema: Asignación de turnos en una empresa.

Una empresa tiene 5 empleados y debe asignarles turnos de trabajo a lo largo de 7 días.
Cada día tiene tres turnos (mañana, tarde y noche), y cada turno necesita exactamente un empleado.
Las reglas son las siguientes:

1. Cada empleado trabaja un máximo de 5 días a la semana.
2. Ningún empleado puede trabajar dos turnos en el mismo día.
3. Cada empleado debe descansar al menos un día entre turnos de noche y mañana.
4. Al menos un turno por semana debe asignarse a cada empleado.
¿Podrás formular y resolver este problema con OR-Tools? 🚀

In [1]:
from ortools.init.python import init
from ortools.linear_solver import pywraplp

In [2]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

In [3]:
num_workers = 5
num_days = 7
num_turns = 3

In [4]:
x = {}
for i in range(num_workers):
    for j in range(num_days):
        for k in range(num_turns):
            x[i, j, k] = solver.IntVar(0, 1, "")

In [5]:
# 1 turno al día por trabajador como máximo
for i in range(num_workers):
    for j in range(num_days):
        solver.Add(solver.Sum([x[i, j, k] for k in range(num_turns)]) == 1)
        
# 1 turno por empleado como máximo
for j in range(num_days):
    for k in range(num_turns):
        solver.Add(solver.Sum([x[i, j, k] for i in range(num_workers)]) == 1)

# Cada empleado trabaja un máximo de 5 días a la semana.
for i in range(num_workers):
    solver.Add(solver.Sum([solver.Sum([x[i, j, k] for k in range(num_turns)]) for j in range(num_days)]) <= 5)

# Ningún empleado trabaja más de 1 turno al día.
for i in range(num_workers):
    for j in range(num_days):
        solver.Add(solver.Sum([x[i, j, k] for k in range(num_turns)]) <= 1)
        
# Cada empleado debe descansar al menos un día entre turnos de noche y mañana.
for i in range(num_workers):
    for j in range(num_days):
        solver.Add(solver.Sum([x[i, j, 2],x[i, int((j+1)%(num_days-1)), 0]]) <= 1)
        
# Al menos un turno por semana debe asignarse a cada empleado.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j, k] for j in range(num_days) for k in range(num_turns)]) >= 1)


In [6]:
objective_terms = []
for i in range(num_workers):
    for j in range(num_days):
        for k in range(num_turns):
            objective_terms.append(x[i, j, k])
solver.Minimize(solver.Sum(objective_terms))

In [7]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

Solving with SCIP 9.0.0 [LP solver: Glop 9.11]


In [8]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")

No solution found.
